In [ ]:
#coding=utf8

import os
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d

#Q1 Global methane levels from 2002
def GML(path):
    ds = xr.open_dataset(path, engine="netcdf4")

    #Q1-1
    xch_clim = ds.xch4.groupby('time.month').mean()
    xch_clim.mean(dim='lon').transpose().plot.contourf(levels=12, robust=True, cmap='Spectral')
    plt.title("Q1-1: Methane climatology for each month")
    plt.show()
    plt.close()

    #Q1-2
    plt.figure(figsize=(9, 6))
    x = ds.xch4.mean(dim=['lon', 'lat'])["time"].values
    y = ds.xch4.mean(dim=['lon', 'lat']).values
    x = np.ma.masked_array(x, mask=np.isnan(y)).compressed()
    y = np.ma.masked_array(y, mask=np.isnan(y)).compressed()
    length = len(y)
    x_fit = np.linspace(1, length, length)
    # 三次样条拟合,且拟合曲线必过所有样本点
    func = interp1d(x_fit, y, kind='linear')
    y_fit = func(x_fit) * 10 ** 9
    plt.tick_params("y", direction="in", length=300, color="silver", left="off", right="off", labelleft="on",
                    labelright="on", labelcolor="silver", labelsize=8)
    plt.tick_params("x", direction="out", color="silver", labelcolor="silver", labelsize=8)
    ax = plt.gca()
    ax.set(ylim=(1600, 1900), xlim=([x[0], x[-1]]))
    ax.yaxis.set_major_formatter(mticker.FormatStrFormatter('%.0f ppb'))
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    plt.plot(x, y_fit, color="brown", linewidth=1)
    plt.fill_between(x=x, y1=1600, y2=y_fit, facecolor='red', alpha=0.3)
    plt.scatter(x, y * 10 ** 9, color="brown", marker=".", s=4)
    plt.title("Q1-2: Global Methane (CH4) - Monthly Means")
    plt.legend(['Trend', '_None', 'Average'])
    plt.show()
    plt.close()

    #Q1-3
    plt.figure(figsize=(9, 6))
    group_data = ds.xch4.groupby('time.month')
    modified = group_data - group_data.mean(dim='time')
    modified.sel(lon=210, lat=-15, method='nearest').plot()
    plt.title("Q1-3: Deseasonalized methane levels at point (15°S, 150°W)")
    plt.show()
    plt.close()

#Q2 Niño 3.4 index
def Nino(path):
    ds = xr.open_dataset(path, engine="netcdf4")

    #Q2-1
    nino_34 = ds.sst.sel(lon=slice(190, 240), lat=slice(-5, 5))
    group_data = nino_34.groupby('time.month')
    nino_34_clim = group_data.mean()
    nino_34_clim.mean(dim=['lat', 'lon']).plot()
    plt.title("Q2-1(1): Monthly climatology for SST from Niño 3.4 region")
    plt.show()
    plt.close()

    # Apply mean to grouped data, and then compute the anomaly
    nino_34_anom = group_data - group_data.mean(dim='time')
    nino_34_anom

    # Plot anomalies
    nino_34_anom.mean(dim=['lat', 'lon']).plot()
    plt.title("Q2-1(2): Anomalies of climatology from  from SST time series")
    plt.show()
    plt.close()

    #Q2-2
    nino_34_roll = nino_34.mean(dim=['lat', 'lon']).rolling(time=3).mean() -27.04
    nino_34_roll = nino_34_roll.to_series()
    nino_34_pos = nino_34_roll.copy()
    nino_34_pos[nino_34_pos < 0] = 0
    nino_34_neg = nino_34_roll.copy()
    nino_34_neg[nino_34_neg > 0] = 0
    nino_all = pd.DataFrame(index=nino_34_roll.index)
    nino_all['neg'] = nino_34_neg
    nino_all['pos'] = nino_34_pos
    nino_all['roll'] = nino_34_roll
    nino_all['pos_thre'] = [0.5] * len(nino_all.index)
    nino_all['neg_thre'] = [-0.5] * len(nino_all.index)
    nino_all['id'] = range(len(nino_all.index))
    nino_all = nino_all.set_index('id')
    fig = plt.figure()
    ax = fig.add_subplot(111)

    # plot 3-month mean sst
    nino_all.roll.plot.line(linewidth=0.8, figsize=(14, 5.3), ax=ax, color='black')

    # plot El Nino & La Nina Threshold
    nino_all.neg_thre.plot.line(linewidth=0.5, ax=ax, color='blue', linestyle='--')
    nino_all.pos_thre.plot.line(linewidth=0.5, ax=ax, color='red', linestyle='--')

    # plot BAR of Anomaly
    nino_all.neg.plot.bar(width=1, stacked=False, ax=ax, color='blue')
    nino_all.pos.plot.bar(width=1, stacked=False, ax=ax, color='red')

    # x_label generating
    years = [int(str(i)[:4]) for i in nino_34_roll.index]
    ax.set_xticklabels(years, rotation=30)
    ax.minorticks_on()
    plt.tick_params("x", which="major", bottom="off", labelbottom="on")
    plt.tick_params("x", which="minor", direction="out", bottom="on", labelbottom="off", top="on", labeltop="off")

    #设置x轴显示12个主刻度, 副刻度等分
    plt.gca().xaxis.set_major_locator(mticker.MultipleLocator(56))
    plt.gca().xaxis.set_minor_locator(mticker.MultipleLocator(28))

    # -----------------------  Figure global parameter adjust -----------------------
    #设置x轴网格线
    plt.grid(which="major", axis="x", linestyle=":")
    plt.title('Q2-2: SST Anomaly in Nino 3.4 Region(5N-5S,120-170W)')
    plt.xlabel('Year')
    plt.ylabel('Anomaly in Degrees C')
    plt.legend(['3mth running mean', 'El Nino Threshold', 'La Nina Threshold'])
    plt.show()
    plt.close()

#Q3 Explore a netCDF dataset
def exploreDataset(path):
    ds = xr.open_dataset(path, engine="netcdf4")

    #Q3-1
    plt.figure(figsize=(10, 8))
    NEE_Mon = ds.NEE.groupby('time.month')
    NEE_Mon_anom = NEE_Mon - NEE_Mon.mean(dim='time')
    plt.subplot(2, 1, 2)
    NEE_Mon_anom.mean(dim=['lat', 'lon']).plot()
    plt.subplot(2, 1, 1)
    NEE_Mon.mean().mean(dim=['lat', 'lon']).plot()
    plt.title("Q3-1: Time series of NEE")
    plt.show()
    plt.close()

    #Q3-2
    print("Q3-2: 5 different plots")
    plt.figure(figsize=(16, 5))
    NEE_Mon_Tol_SUM = ds.NEE.groupby('time.month').sum().sum(dim=('lat', 'lon'))
    ax = plt.subplot(1, 3, 1)
    NEE_Mon_Tol_SUM.plot()
    ax.set_title('PLOT of Monthly Sum NEE')
    ax = plt.subplot(1, 3, 2)
    NEE_Mon_Tol_SUM.to_series().plot(kind='bar')
    ax.set_title('BAR of Monthly Sum NEE')
    ax = plt.subplot(1, 3, 3)
    ds.NEE.plot()
    ax.set_title('Histograms for NEE')
    plt.show()
    plt.close()

    plt.figure(figsize=(9, 16))
    ax = plt.subplot(2, 1, 1)
    ds.NEE.mean(dim='time').plot(robust=True)
    ax.set_title('2D Mesh for Daily-Mean NEE')
    ax = plt.subplot(2, 1, 2)
    ds.NEE.groupby('time.month').sum().mean(dim='month').plot(robust=True)
    ax.set_title(' 2DMesh of Monthly-Mean NEE')
    plt.show()
    plt.close()

if __name__=="__main__":
    #Q1 Global methane levels from 2002
    path = os.path.join(os.getcwd(), "200301_202006-C3S-L3_GHG-PRODUCTS-OBS4MIPS-MERGED-v4.3.nc")
    GML(path)

    #Q2 Niño 3.4 index
    path = os.path.join(os.getcwd(), "NOAA_NCDC_ERSST_v3b_SST.nc")
    Nino(path)

    #Q3 Explore a netCDF dataset
    path = os.path.join(os.getcwd(), "CMSFluxNEE_2012_v1.nc4")
    exploreDataset(path)